# Dependencies Imported

In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt
from scipy import stats


# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# To Reflect an existing database into a new model

Base=automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables


In [7]:
# To view classes automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Saving references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [9]:
# To create session link from Python to DB
session=Session(engine)

In [10]:
#To Inspect engine
inspector=inspect(engine)

# Exploratory Climate Analysis

# Station Table

In [11]:
stations=engine.execute('SELECT * FROM Station')

In [23]:
print(stations.keys())

RMKeyView(['id', 'station', 'name', 'latitude', 'longitude', 'elevation'])


In [24]:
stations.fetchall()

[]

In [25]:
columns=inspector.get_columns('station')
for column in columns:
    print(column['name'], column['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Measurement

In [26]:
measurements=engine.execute('SELECT * FROM Measurement LIMIT 15')

In [27]:
measurements.fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (10, 'USC00519397', '2010-01-11', 0.01, 64.0),
 (11, 'USC00519397', '2010-01-12', 0.0, 61.0),
 (12, 'USC00519397', '2010-01-14', 0.0, 66.0),
 (13, 'USC00519397', '2010-01-15', 0.0, 65.0),
 (14, 'USC00519397', '2010-01-16', 0.0, 68.0),
 (15, 'USC00519397', '2010-01-17', 0.0, 64.0)]

In [28]:
columns=inspector.get_columns('measurement')
for column in columns:
    print(column['name'], column['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Climate Analysis

In [29]:
# To Design a query to retrieve the last 12 months of precipitation data and plot the results

In [30]:
# Calculate the date 1 year ago from the last data point in the database
latest_date=(session.query(measurement.date)
                    .order_by(measurement.date.desc())
                    .first())
latest_date

('2017-08-23',)